In [1]:
import openpyxl
import pandas as pd

In [73]:
class CellMaker :
    
    def __init__(self, filename):
        
        self.filename = filename
        self.input_file = str(self.filename) + '.xlsx'
        self.output_file = str(self.filename) + '_완료.xlsx'
        self.year = self.filename[:4]
        self.file = pd.DataFrame(
                        pd.read_excel(self.input_file,
                             names = ['대상업무','업무량','부담금'],
                             header = None
                            )
                    ).dropna(subset=["대상업무"])

    def isIns(self,cell) :
        if (('Ⅰ' in cell) or ('Ⅱ' in cell) or ('Ⅲ' in cell)) : return True
        else : return False

    def defIns(self,cell) :
        if('한국원자력안전기술' in cell) : return 'KINS'
        elif('한국원자력통제기술' in cell) : return 'KINAC'
        elif('한국원자력의학' in cell) : return 'KIRAMS'
        else : return 'UI'
    
    def depthDefiner(self,cell):

        d1 = ['1.','2.','3.','4.','5.','6.','7.','8.','9.','10.','11.','12.','피규제기관']
        d2 = ['가.','나.','다.','라.','마.','바.','사.','아.','자.','차.','카.','타.','파.','하.','depth1']
        d3 = ['1)','2)','3)','4)','5)','6)','7)','8)','9)','10)','11)','12)','depth2']
        d4 = ['가)','나)','다)','라)','마)','바)','사)','아)','자)','차)','카)','타)','파)','하)','depth3']
        d5 = ['(1)','(2)','(3)','(4)','(5)','(6)','(7)','(8)','(9)','(10)','(11)','(12)','depth4']
        d6 = ['(가)','(나)','(다)','(라)','(마)','(바)','(사)','(아)','(자)','(차)','(카)','(타)','(파)','(하)','depth5']
        d7 = ['①','②','③','④','⑤','⑥','⑦','⑧','⑨','⑩','depth6']
        d8 = ['o','depth7']
        d9 = ['합','삭','etc']

        depthes = [d5,d6,d4,d1,d2,d3,d7,d8,d9]

        for depth in depthes :
            for index in depth :
                if (index in cell.lstrip()[:3]):
                    return depth[-1]

    def tableMaker(self,df) :
        
        table = pd.DataFrame(columns=['년도', '규제기관','피규제기관','대상업무','깊이','업무량','부담금'])

        for i in df.index :
            
            if self.isIns(df['대상업무'][i]) :
                ins = self.defIns(df['대상업무'][i])
            else : 
                if (self.depthDefiner(df['대상업무'][i]) == '피규제기관'):
                    insd = df['대상업무'][i]
                elif (self.depthDefiner(df['대상업무'][i]) == 'etc'):
                    continue
                else :
                    new_data = {
                        '년도' : self.year,
                        '규제기관' : ins,
                        '피규제기관' : insd,
                        '대상업무' : df['대상업무'][i],
                        '깊이' : self.depthDefiner(df['대상업무'][i]),
                        '업무량' : df['업무량'][i],
                        '부담금' : df['부담금'][i]
                    }

                    if (type(new_data['업무량']) == str) : 
                        new_data['업무량'] = 0
                        new_data['부담금'] = 0
                        
                    table = table.append(new_data, ignore_index = True)

        table = table.fillna(0)

        return table
            
    def cellMaker(self,table) :

        number_of_works = len(table['깊이'].unique())
        result = pd.DataFrame(columns=['년도', '규제기관','피규제기관','업무량','부담금'])
        point = 3

        for _ in range(number_of_works):
            name = '대상업무'+ str(_+1)
            result.insert(point,name,'')
            point += 1

        for index in table.index :
            
            if (table['깊이'][index][-1] == '1' and index == 0) :

                new_data = {
                    '년도' : table['년도'][index],
                    '규제기관' : table['규제기관'][index],
                    '피규제기관' : table['피규제기관'][index],
                    '대상업무1' : table['대상업무'][index],
                    '업무량' : table['업무량'][index],
                    '부담금' : table['부담금'][index]
                }

            else :

                if (int(table['깊이'][index - 1][-1]) < int(table['깊이'][index][-1])):

                    if(table['깊이'][index][-1] == '7') :
                        target = '대상업무' + str(int(table['깊이'][index - 1][-1]) + 1)
                        sp_target = target
                    else : 
                        target = '대상업무' + table['깊이'][index][-1]

                    new_data[target] = table['대상업무'][index]
                    new_data['업무량'] = table['업무량'][index]
                    new_data['부담금'] = table['부담금'][index]

                else :
                    result = result.append(new_data, ignore_index = True)

                    new_data = {
                    '년도' : table['년도'][index],
                    '규제기관' : table['규제기관'][index],
                    '피규제기관' : table['피규제기관'][index],
                    }

                    if(table['깊이'][index][-1] == '7') :
                        target = sp_target
                    else : 
                        target = '대상업무' + table['깊이'][index][-1]

                    new_data[target] = table['대상업무'][index]
                    new_data['업무량'] = table['업무량'][index]
                    new_data['부담금'] = table['부담금'][index]

            if (index == table.index[-1]):
                result = result.append(new_data, ignore_index = True)
        
        result.to_excel(self.output_file, index = False)

In [74]:
to_do_list = [ 
              '2022부담금','2022부담금외',
              '2021부담금','2021부담금외',
              '2020부담금','2020부담금외',
              '2019부담금','2019부담금외',
              '2018부담금','2018부담금외',
              '2017부담금','2017부담금외',
              '2016부담금','2016부담금외',              
              '2022비용','2022비용외',
              '2021비용','2021비용외',
              '2020비용','2020비용외',
              '2019비용','2019비용외',
              '2018비용','2018비용외',
              '2017비용','2017비용외',
              '2016비용','2016비용외']

'''
              '2022부담금','2022부담금외'
              '2021부담금','2021부담금외',
              '2020부담금','2020부담금외',
              '2019부담금','2019부담금외',
              '2018부담금','2018부담금외',
              '2017부담금','2017부담금외',
              '2016부담금','2016부담금외'
'''


for document in to_do_list :
    print(document + " 작업중..." , end = '')
    a = CellMaker(document)
    a.cellMaker(a.tableMaker(a.file))
    print(' 완료!')
print('Done..')

2022부담금 작업중... 완료!
2022부담금외 작업중... 완료!
2021부담금 작업중... 완료!
2021부담금외 작업중... 완료!
2020부담금 작업중... 완료!
2020부담금외 작업중... 완료!
2019부담금 작업중... 완료!
2019부담금외 작업중... 완료!
2018부담금 작업중... 완료!
2018부담금외 작업중... 완료!
2017부담금 작업중... 완료!
2017부담금외 작업중... 완료!
2016부담금 작업중... 완료!
2016부담금외 작업중... 완료!
2022비용 작업중... 완료!
2022비용외 작업중... 완료!
2021비용 작업중... 완료!
2021비용외 작업중... 완료!
2020비용 작업중... 완료!
2020비용외 작업중... 완료!
2019비용 작업중... 완료!
2019비용외 작업중... 완료!
2018비용 작업중... 완료!
2018비용외 작업중... 완료!
2017비용 작업중... 완료!
2017비용외 작업중... 완료!
2016비용 작업중... 완료!
2016비용외 작업중... 완료!
Done..


In [70]:
a = CellMaker('2022비용외')
a.tableMaker(a.file)

,년도,규제기관,피규제기관,대상업무,깊이,업무량,부담금
0,2022,KINS,1. 한국원자력연구원,가. 법 제20조제1항 및 제37조제3항에 따른 승인에 관련된 심사,depth1,100,40050000
1,2022,KINS,1. 한국원자력연구원,나 법제36조제1항에 따른 교육에 관한 관리업무,0,0,0
2,2022,KINS,1. 한국원자력연구원,다. 법 제37조제4항에 따른 방사능방재 훈련평가 지원(방사선비상진료에 관한 사항은...,depth1,80,32040000
3,2022,KINS,1. 한국원자력연구원,라. 법 제38조제1항에 따른 검사(방사선비상진료에 관한 사항은 제외),depth1,125,50062000
4,2022,KINAC,1. 한국원자력연구원,가. 법 제4조 제1항에 따른 원자력시설 등에 대한 위협의 평가,depth1,0,0
5,2022,KINAC,1. 한국원자력연구원,1) 위협평가 및 설계기준위협 재설정(물리적방호),depth2,45,14436000
6,2022,KINAC,1. 한국원자력연구원,2) 위협평가 및 설계기준위협 재설정(사이버보안),depth2,23,7378000
7,2022,KINAC,1. 한국원자력연구원,나. 법 제9조 제1항에 따른 승인에 관련된 심사,depth1,0,0
8,2022,KINAC,1. 한국원자력연구원,1) 최초심사 및 변경심사(물리적방호),depth2,64,20531000
9,2022,KINAC,1. 한국원자력연구원,2) 최초심사 및 변경심사(사이버보안),depth2,144,46195000


In [53]:
pd.set_option('display.max_rows', None)